In [7]:
import numpy as np
import os
import pandas as pd
import tf.transformations as tr
from tqdm import tqdm

In [4]:
path_to_tfs = '/home/mik/Desktop/drawing_data_one_direction/blue_marker/tfs'

file_names = [f for f in os.listdir(path_to_tfs) if os.path.isfile(os.path.join(path_to_tfs, f))]

In [10]:
def df_to_tfs(all_tfs):
    converted_all_tfs = {}
    parent_frame = all_tfs['parent_frame'][0] # Assume that are all teh same
    child_frames = all_tfs['child_frame']
    converted_all_tfs[parent_frame] = np.eye(4) # Transformation to itself is the identity
    all_poses = all_tfs[['x','y','z','qx','qy','qz','qw']]
    for i, child_frame_i in enumerate(child_frames):
        pose_i = all_poses.iloc[i]
        X_i = tr.quaternion_matrix(pose_i[3:])
        X_i[:3,3] = pose_i[:3]
        converted_all_tfs[child_frame_i] = X_i
    return converted_all_tfs

In [18]:
def get_transformation_matrix(all_tfs, source_frame, target_frame):
    w_X_sf = all_tfs[source_frame]
    w_X_tf = all_tfs[target_frame]
    sf_X_w = np.linalg.inv(w_X_sf)
    sf_X_tf = sf_X_w @ w_X_tf
    return sf_X_tf

In [19]:
def pose_to_X(pose):
    pos = pose[:3]
    quat = pose[3:]
    X = tr.quaternion_matrix(quat)
    X[:3,3] = pos
    return X

def X_to_pose(X):
    pos = X[:3,3]
    quat = tr.quaternion_from_matrix(X)
    pose = np.concatenate([pos, quat])
    return pose

In [29]:
for fn in tqdm(file_names):
    path_file_i = os.path.join(path_to_tfs, fn)
    df_i = pd.read_csv(path_file_i)
    all_tfs_i = df_to_tfs(df_i)
    # compute the missing tf
    w_X_gf = all_tfs_i['grasp_frame']
    gf_X_fl = get_transformation_matrix(all_tfs_i, 'grasp_frame', 'wsg50_finger_left')
    current_half_width = -gf_X_fl[...,0,3]-0.009
    gf_pose_fr_zero_gw = np.array([0.009, 0, -0.105, 0, 0, 1, 0])
    gf_X_fr_zero_gw = pose_to_X(gf_pose_fr_zero_gw)
    fr_X_fr_new = np.eye(4)
    fr_X_fr_new[0,3] = -current_half_width  # translation along -x
    gf_X_fr = gf_X_fr_zero_gw @ fr_X_fr_new
    w_X_fr = w_X_gf @ gf_X_fr
    fr_pose = X_to_pose(w_X_fr)
    df_to_add = pd.DataFrame([['med_base', 'wsg50_finger_right'] + list(fr_pose)], columns=df_i.columns)
    updated_df = df_i.append(df_to_add)
    updated_df.to_csv(path_file_i, index=False) # Overwrite the file
    
    
    

100%|████████████████████████████████| 1471/1471 [00:04<00:00, 366.28it/s]


In [26]:
updated_df

,parent_frame,child_frame,x,y,z,qx,qy,qz,qw
0,med_base,grasp_frame,0.634863,0.183552,0.102341,-0.845579,0.533172,0.024368,-0.011424
1,med_base,med_kuka_link_ee,0.645755,0.182193,0.306046,-0.220903,0.974924,0.009152,-0.025309
2,med_base,wsg50_finger_left,0.632061,0.200485,0.207756,-0.845579,0.533172,0.024368,-0.011424
3,med_base,pico_flexx_left_link,0.604876,0.252132,0.158595,-0.719599,0.069557,-0.591591,0.356874
4,med_base,pico_flexx_left_optical_frame,0.604876,0.252132,0.158595,-0.761181,0.467503,-0.369134,-0.256485
5,med_base,pico_flexx_right_link,0.670630,0.114251,0.154159,0.046459,0.689349,-0.355276,-0.629618
6,med_base,pico_flexx_right_optical_frame,0.670630,0.114251,0.154159,0.478059,0.738661,0.236225,-0.412356
0,med_base,wsg50_finger_right,0.648878,0.165221,0.206622,0.533172,0.845579,-0.011424,-0.024368


In [24]:
df_i.columns

Index(['parent_frame', 'child_frame', 'x', 'y', 'z', 'qx', 'qy', 'qz', 'qw'], dtype='object')